# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from joblib import dump, load
import time

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
data.head(10)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Целевым столбцом является `is_ultra`, все остальные отнесем к признакам.

In [5]:
data.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

## Разбейте данные на выборки

In [6]:
inter_df, df_test = train_test_split(data, test_size=0.2, random_state=333)
df_train, df_valid = train_test_split(inter_df, test_size=0.25, random_state=333)

def tar_feat_div(df):
    feat =  df.drop('is_ultra', axis=1)
    tar = df['is_ultra']
    return feat, tar

features_train, target_train = tar_feat_div(df_train)
features_test, target_test = tar_feat_div(df_test)
features_valid, target_valid = tar_feat_div(df_valid)

Получили три набора данных, разбитых в соотношении 60% : 20% : 20% относительно начального набора данных. 

## Исследуйте модели

**Рассмотри модели решающего дерева различной глубины.**

In [7]:
best_model_tree = None
best_result = 0
for depth in range(1, 6):
    start = time.time()
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model_tree.fit(features_train, target_train)
    predictions_tree = model_tree.predict(features_valid)
    result_tree = accuracy_score(target_valid, predictions_tree)
    end = time.time()
    if result_tree > best_result:
        best_model_tree = model_tree
        best_depth = depth
        best_result = result_tree
        time_tree = end - start
dump(best_model_tree, 'best_model_tree.joblib')

print("Accuracy лучшей модели:", best_result, 'лучшая глубина:', best_depth)
print('Время выполнения программы:', time_tree)

Accuracy лучшей модели: 0.7947122861586314 лучшая глубина: 5
Время выполнения программы: 0.0064792633056640625


**Рассмотри модели случайного леса с различными гиперпараметрами.**

In [8]:
best_model_forest = None
best_result = 0
for est in range(10, 51, 10):
    for depth_f in range (1, 11):
        start = time.time()
        model_forest = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth_f)
        model_forest.fit(features_train, target_train)
        predictions_forest = model_forest.predict(features_valid)
        result_forest = accuracy_score(target_valid, predictions_forest)
        end = time.time()
        if result_forest > best_result:
            best_model_forest = model_forest
            best_result = result_forest
            best_est = est
            best_depth = depth_f
            time_forest = end - start
dump(best_model_forest, 'best_model_forest.joblib')

print("Accuracy лучшей модели:", best_result, "Количество деревьев:", best_est, "Максимальная глубина:", best_depth)
print('Время выполнения программы:', time_forest)

Accuracy лучшей модели: 0.8211508553654744 Количество деревьев: 50 Максимальная глубина: 9
Время выполнения программы: 0.17224693298339844


**Рассмотри модель логистической регрессии.**

In [9]:
start = time.time()
model_reg = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model_reg.fit(features_train, target_train)
predictions_reg = model_reg.predict(features_valid)
result_reg = accuracy_score(target_valid, predictions_reg)
end = time.time()
dump(model_reg, 'model_reg.joblib')

print("Accuracy лучшей модели:", result_reg)
print('Время выполнения программы:', end-start)

Accuracy лучшей модели: 0.7465007776049767
Время выполнения программы: 0.03337979316711426


**Вывод: модель решающего дерева работает быстрее всего, но точность меньше чем у логической регрессии и случайного леса. Случайный лес получает самые точные значения, но работает дольше всего. Все модели получили точность более 0.75. Проверим все три модели на тестовой выборке**

## Проверьте модель на тестовой выборке

In [10]:
best_model_forest = load('best_model_forest.joblib')
predictions_forest = best_model_forest.predict(features_test)
forest_accuracy = accuracy_score(target_test, predictions_forest)

print("Accuracy случайного леса:", forest_accuracy)

Accuracy случайного леса: 0.8211508553654744


**Видим, что после проверки на тестовом наборе лучше всего себя показала модель случайного леса из 50 деревьев с максимальной глубиной дерева 9, ей мы и отдадим предпочтение.**